In [10]:
#load packages
import numpy as np
import pandas as pd
import dtale
from hmmlearn import hmm
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from scipy.optimize import linear_sum_assignment
from jumpmodels.sparse_jump import SparseJumpModel    # Sparse JM class
from jumpmodels.jump import JumpModel                 # JM class

In [11]:
def simulate_data(T, P, mu, random_state=None):
    """
    Simulate data from a 2-state Gaussian HMM.
    
    Parameters:
        T (int): Number of observations.
        P (int): Total number of features (only first 15 are informative).
        mu (float): Signal magnitude for informative features.
        random_state (int or None): Seed for reproducibility.
        
    Returns:
        X (ndarray): Simulated observations (T x P).
        states (ndarray): True state sequence (length T).
    """
    rng = np.random.default_rng(random_state)
    
    # Transition matrix for 2-state HMM with higher switching probability.
    transmat = np.array([[0.9979, 0.0021],
                         [0.0120, 0.9880]])
    
    # Compute stationary distribution (eigenvector corresponding to eigenvalue 1)
    eigvals, eigvecs = np.linalg.eig(transmat.T)
    stat = np.real(eigvecs[:, np.isclose(eigvals, 1)])
    stat = stat[:, 0]
    stat = stat / np.sum(stat)
    
    # Generate state sequence
    states = np.zeros(T, dtype=int)
    states[0] = rng.choice(np.arange(2), p=stat)
    for t in range(1, T):
        states[t] = rng.choice(np.arange(2), p=transmat[states[t-1]])
    
    # Define state means: state 0 has mean mu for first 15 features,
    # state 1 has mean -mu for first 15 features.
    means = np.zeros((2, P))
    if P >= 15:
        means[0, :15] = -mu
        means[1, :15] = mu
    else:
        means[0, :P] = -mu
        means[1, :P] = mu
    
    # Generate observations: each observation is drawn from N(mean[state], I_P)
    X = np.zeros((T, P))
    for t in range(T):
        X[t] = rng.normal(loc=means[states[t]], scale=1.0, size=P)
    
    return X, states

# Simulate data
T = 500  # Number of observations for display purposes
P = 50   # Number of features for display purposes  
mu = 1.0
X, states = simulate_data(T, P, mu, random_state=1)

# Create a DataFrame to display the simulated data and states
df_simulated = pd.DataFrame(X, columns=[f'Feature_{i+1}' for i in range(P)])
df_simulated['State'] = states
df_simulated.index.name = 'Time'


df_simulated.head()


,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_42,Feature_43,Feature_44,Feature_45,Feature_46,Feature_47,Feature_48,Feature_49,Feature_50,State
Time,,,,,,,,,,,,,,,,,,,,,
0,-0.827144,-1.379214,-0.438970,-3.135831,-0.767627,-0.971874,-2.370340,1.175598,-2.387413,-2.077520,...,-1.246077,0.567380,1.874081,1.197054,0.991107,0.020178,0.983631,-0.966133,0.751015,0
1,-1.087248,0.130906,-0.533989,-2.089880,-0.872007,0.218221,-2.129234,-1.559140,-1.768673,-2.495646,...,0.851642,-0.914445,-2.729085,-1.059957,0.094160,-3.080178,-0.357138,-0.332090,-1.426456,0
2,-2.482503,-1.462462,-1.547100,0.259919,-0.625137,-2.580855,-1.854651,-0.294858,0.895977,-0.613501,...,-0.912098,-0.476830,-0.351141,0.660808,0.137527,0.402087,0.595284,0.029938,1.110566,0
3,-1.728959,-2.436121,-2.776872,-2.621010,-1.550516,0.030047,-0.825833,-1.598547,-2.069856,-1.528886,...,1.432875,-0.822446,-0.476102,0.839665,-2.743717,-1.066791,1.033177,0.465109,0.978769,0
4,-1.309761,-0.436001,-1.730844,-2.371960,-2.694795,-1.551710,-1.548941,-1.629377,-4.548805,-2.998604,...,0.447018,0.268373,-1.107013,0.585332,-1.741727,-0.178998,-0.630314,-0.529276,-1.083458,0




## Align Labels Function

This function aligns predicted labels with true labels using the Hungarian algorithm.



In [12]:

df_simulated.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_42,Feature_43,Feature_44,Feature_45,Feature_46,Feature_47,Feature_48,Feature_49,Feature_50,State
Time,,,,,,,,,,,,,,,,,,,,,
0,-0.827144,-1.379214,-0.438970,-3.135831,-0.767627,-0.971874,-2.370340,1.175598,-2.387413,-2.077520,...,-1.246077,0.567380,1.874081,1.197054,0.991107,0.020178,0.983631,-0.966133,0.751015,0
1,-1.087248,0.130906,-0.533989,-2.089880,-0.872007,0.218221,-2.129234,-1.559140,-1.768673,-2.495646,...,0.851642,-0.914445,-2.729085,-1.059957,0.094160,-3.080178,-0.357138,-0.332090,-1.426456,0
2,-2.482503,-1.462462,-1.547100,0.259919,-0.625137,-2.580855,-1.854651,-0.294858,0.895977,-0.613501,...,-0.912098,-0.476830,-0.351141,0.660808,0.137527,0.402087,0.595284,0.029938,1.110566,0
3,-1.728959,-2.436121,-2.776872,-2.621010,-1.550516,0.030047,-0.825833,-1.598547,-2.069856,-1.528886,...,1.432875,-0.822446,-0.476102,0.839665,-2.743717,-1.066791,1.033177,0.465109,0.978769,0
4,-1.309761,-0.436001,-1.730844,-2.371960,-2.694795,-1.551710,-1.548941,-1.629377,-4.548805,-2.998604,...,0.447018,0.268373,-1.107013,0.585332,-1.741727,-0.178998,-0.630314,-0.529276,-1.083458,0


In [13]:
def align_labels(true_labels, pred_labels):
    """
    Align predicted labels with true labels using the Hungarian algorithm.
    
    Returns:
        aligned (ndarray): Predicted labels after optimal permutation.
    """
    D = confusion_matrix(true_labels, pred_labels)
    row_ind, col_ind = linear_sum_assignment(-D)
    mapping = {col: row for row, col in zip(row_ind, col_ind)}
    aligned = np.array([mapping[x] for x in pred_labels])
    return aligned



## Run HMM Function

This function fits a Gaussian HMM to the data using `hmmlearn`.



In [14]:
def run_hmm(X, n_components=2, random_state=None):
    """
    Fit a Gaussian HMM to the data X using hmmlearn.
    
    Parameters:
        X (ndarray): Observations (T x P).
        n_components (int): Number of hidden states.
        random_state (int or None): Seed for reproducibility.
        
    Returns:
        pred_states (ndarray): Inferred state sequence.
    """
    # Let the HMM initialize all parameters by itself (i.e., start probabilities, transition matrix, means, covariances)
    model = hmm.GaussianHMM(n_components=n_components, covariance_type='diag',
                            n_iter=100, random_state=random_state)
    model.fit(X)
    pred_states = model.predict(X)
    return pred_states

# Example usage:
pred_states = run_hmm(X, random_state=42)



## Calculate Balanced Accuracy Function

This function computes the Balanced Accuracy (BAC) after aligning the predicted state labels.



In [15]:
def calculate_bac(true_states, pred_states):
    """
    Compute the Balanced Accuracy (BAC) after aligning the predicted state labels.
    """
    aligned_pred = align_labels(true_states, pred_states)
    bac = balanced_accuracy_score(true_states, aligned_pred)
    return bac



## Main Execution

This section runs the simulation, fits the HMM, and calculates the BAC for different numbers of features.



In [16]:
if __name__ == '__main__':
    # Simulation settings
    T = 500
    mu_values = [0.125, 0.250, 0.375, 0.5, 1]  # Different signal magnitudes
    p_values = [15, 30, 60, 150, 300]  # Different numbers of features
    n_simulations = 10  # Increase to 100 for full replication
    
    results = []
    
    for mu in mu_values:
        for P in p_values:
            bac_hmm_list = []
            bac_jump_normal_list = []
            bac_sparse_jump_list = []
            
            for sim in range(n_simulations):
                # Simulate data and get true state sequence
                X, true_states = simulate_data(T, P, mu, random_state=sim)
                
                # ----- HMM Analysis -----
                pred_states_hmm = run_hmm(X, random_state=sim)
                bac_hmm = calculate_bac(true_states, pred_states_hmm)
                bac_hmm_list.append(bac_hmm)
                
                # ----- Normal (Standard) Jump Model Analysis -----
                # Use the jump penalty value from Nystrup et al.: lambda = 400
                jump_model_normal = JumpModel(n_components=2, jump_penalty=30, cont=False, )
                jump_model_normal.fit(X)
                pred_states_jump_normal = jump_model_normal.labels_
                bac_jump_normal = calculate_bac(true_states, pred_states_jump_normal)
                bac_jump_normal_list.append(bac_jump_normal)
                
                # ----- Sparse Jump Model Analysis -----
                # For sparse jump model, use lambda = 30 and kappa = 17 (i.e., max_feats = 300)
                sparse_jump_model = SparseJumpModel(n_components=2, max_feats=50, jump_penalty=30, random_state=sim)
                sparse_jump_model.fit(X)
                pred_states_sparse_jump = sparse_jump_model.labels_
                bac_sparse_jump = calculate_bac(true_states, pred_states_sparse_jump)
                bac_sparse_jump_list.append(bac_sparse_jump)
            
            mean_bac_hmm = np.mean(bac_hmm_list)
            mean_bac_jump_normal = np.mean(bac_jump_normal_list)
            mean_bac_sparse_jump = np.mean(bac_sparse_jump_list)
            
            results.append({
                'mu': mu, 
                'P': P, 
                'HMM Mean BAC': mean_bac_hmm,
                'Normal Jump Model Mean BAC': mean_bac_jump_normal,
                'Sparse Jump Model Mean BAC': mean_bac_sparse_jump
            })
            print(f"mu = {mu}, P = {P}, HMM BAC = {mean_bac_hmm:.3f}, Normal Jump BAC = {mean_bac_jump_normal:.3f}, Sparse Jump BAC = {mean_bac_sparse_jump:.3f}")
    


/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/g

mu = 0.125, P = 15, HMM BAC = 0.594, Normal Jump BAC = 0.637, Sparse Jump BAC = 0.687


/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/g

mu = 0.125, P = 30, HMM BAC = 0.563, Normal Jump BAC = 0.731, Sparse Jump BAC = 0.691


/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:409: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.

/Users/vlad/Desktop/g

...

mu = 1, P = 60, HMM BAC = 0.961, Normal Jump BAC = 1.000, Sparse Jump BAC = 1.000


/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true



mu = 1, P = 150, HMM BAC = 0.939, Normal Jump BAC = 0.955, Sparse Jump BAC = 0.959


/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning:

y_pred contains classes not in y_true



mu = 1, P = 300, HMM BAC = 0.861, Normal Jump BAC = 0.999, Sparse Jump BAC = 1.000


In [17]:
  df_results = pd.DataFrame(results)
  print(df_results)

       mu    P  HMM Mean BAC  Normal Jump Model Mean BAC  Sparse Jump Model Mean BAC
0   0.125   15      0.594136                    0.637420                    0.686885
1   0.125   30      0.562618                    0.731473                    0.690945
2   0.125   60      0.555005                    0.726603                    0.673164
3   0.125  150      0.511685                    0.629139                    0.462996
4   0.125  300      0.503963                    0.579301                    0.649415
5   0.250   15      0.794359                    0.898283                    0.897662
6   0.250   30      0.766352                    0.798029                    0.797699
7   0.250   60      0.679901                    0.782458                    0.816550
8   0.250  150      0.586163                    0.858388                    0.791063
9   0.250  300      0.514988                    0.728705                    0.745214
10  0.375   15      0.844187                    0.930194         